In [1]:
import pandas as pd
import glob as gb
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
files = gb.glob("/home/rodriguesms/Dados/SINAN/Dados/HANS/*.txt")
files

['/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR14.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR19.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR21.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR22.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR12.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR20.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR13.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR18.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR10.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR16.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR15.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR17.txt',
 '/home/rodriguesms/Dados/SINAN/Dados/HANS/HANSBR11.txt']

In [3]:
dataFrames = [pd.read_csv(f, sep = " ", encoding = "iso-8859-1", low_memory = False) for f in files]

In [4]:
df = pd.concat(dataFrames, ignore_index = True)
df.head()

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_DIAG,SEM_DIAG,...,CLASSATUAL,AVAL_ATU_N,ESQ_ATU_N,DOSE_RECEB,EPIS_RACIO,DTMUDESQ,CONTEXAM,DTALTA_N,TPALTA_N,IN_VINCULA
0,2,A309,2014-01-01,2014,29.0,291560,1388.0,5504120.0,2013-12-10,NaN,...,1.0,0.0,1.0,6.0,4.0,NaN,1.0,2014-09-08,1.0,NaN
1,2,A309,2014-01-01,2014,29.0,291560,1388.0,5504120.0,2014-01-01,NaN,...,1.0,NaN,1.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN
2,2,A309,2014-01-01,2014,50.0,500270,1975.0,10154.0,2014-01-01,NaN,...,2.0,0.0,2.0,23.0,4.0,NaN,1.0,2016-10-05,1.0,NaN
3,2,A309,2014-01-01,2014,21.0,211230,1443.0,2450712.0,2014-01-01,NaN,...,2.0,NaN,2.0,12.0,4.0,NaN,3.0,2016-02-05,1.0,NaN
4,2,A309,2014-01-02,2014,29.0,291735,1403.0,6352960.0,2013-12-27,NaN,...,2.0,0.0,2.0,12.0,4.0,NaN,8.0,2015-02-20,1.0,NaN


In [5]:
#lower case columns
df.columns = df.columns.str.lower()

In [6]:
#select only new cases (1 new case)
df.query("modoentr == 1", inplace = True)

In [7]:
#select only cases that occurs til december 2022
df.query("nu_ano >= 2011", inplace = True)

In [8]:
dic = {
     "cs_raca": {1: "White", 2: "Non white", 3: "Non white", 4: "Non white", 5:"Non white"},
     "cs_sexo": {"F": "Female", "M": "Male"},
     "cs_escol_n": {1: "< 5 years", 2:"[5,9)", 3:"[5,9)", 4:"[9,12)", 5:"[9,12)", 6:"[9,12)", 7:">=12", 8:">=12"},
     "tpalta_n": {1: "Non death", 2: "Non death", 3:"Non death", 4:"Non death", 5: "Non death", 6: "Death", 7: "Non death", 8: "Non death", 9: "Non death"},
     "cs_zona": {1: "Urban", 2: "Rural", 3: "Peri-urban"},
     "baciloscop": {1:"Positive", 2: "Negative"},
     "avalia_n": {0: "Zero Degree", 1: "First Degree", 2: "Second Degree"},
     "aval_atu_n":{0: "Zero Degree", 1: "First Degree", 2: "Second Degree"},
     "classatual":{1: "PB", 2: "MB"},
     "formaclini": {2: "Tuberculoid", 3: "Bordline", 4: "Virchowiana"},
     "classopera": {1: "PB", 2:"MB"},
     "modoentr": {1: "New Case", 2:"Transfer", 3: "Transfer", 4:"Transfer", 5:"Transfer", 6:"Recidiva"},
     "bacilosco": {1: "Positive", 2: "Negative", 3:"Not performed"},
     "esq_ini_n": {1: "PQT/PB/6 doses", 2: "PQT/MB/12 doses", 3: "Others"},
     "esq_atu_n": {1: "PQT/PB/6 doses", 2: "PQT/MB/12 doses", 3: "Others"},
     "tpalta_n": {1: "Cure", 2: "transfer", 3:"transfer", 4:"transfer", 5:"transfer", 6:"Death", 7:"Leaft treatment", 8: "Miss Diagnosis", 9: "transfer"},
     "epis_racio": {1:"Type 1", 2:"Type 2", 3: "Booth"}
}


for col in df.columns:
    if col in dic.keys():
        df[col] = df[col].map(dic[col])

In [9]:
#Convert age to numeric
def convertAge(x):
    """
    input: a integer with 4 digits
    return: age in years
    """
    x = str(x)
    x = x.split(".")[0]
    try:
        if x[0] == "4":
            return int(x[1:])
        elif x[0] == "3":
            return int(x[1:])/12
        elif x[0] == "2":
            return int(x[1:])/365.5
        else:
            return np.nan
    except:
        return np.nan
df["age"] = df["nu_idade_n"].apply(lambda x: convertAge(x))

In [10]:
names = ["PB", "MB", "overall"]
for name in names:
    if name == "overall":
        profile = ProfileReport(df, minimal = True)
        profile.to_file("../results/" + name + "_Descriptive.html")
    else:
         profile = ProfileReport(df[df["classopera"] == name], minimal = True)
         profile.to_file("../results/" + name + "_Descriptive.html")

/usr/local/lib/python3.10/dist-packages/multimethod/__init__.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
df.head()

,tp_not,id_agravo,dt_notific,nu_ano,sg_uf_not,id_municip,id_regiona,id_unidade,dt_diag,sem_diag,...,aval_atu_n,esq_atu_n,dose_receb,epis_racio,dtmudesq,contexam,dtalta_n,tpalta_n,in_vincula,age
0,2,A309,2014-01-01,2014,29.0,291560,1388.0,5504120.0,2013-12-10,NaN,...,Zero Degree,PQT/PB/6 doses,6.0,NaN,NaN,1.0,2014-09-08,Cure,NaN,25.0
1,2,A309,2014-01-01,2014,29.0,291560,1388.0,5504120.0,2014-01-01,NaN,...,NaN,PQT/PB/6 doses,6.0,NaN,NaN,NaN,NaN,NaN,NaN,78.0
2,2,A309,2014-01-01,2014,50.0,500270,1975.0,10154.0,2014-01-01,NaN,...,Zero Degree,PQT/MB/12 doses,23.0,NaN,NaN,1.0,2016-10-05,Cure,NaN,53.0
3,2,A309,2014-01-01,2014,21.0,211230,1443.0,2450712.0,2014-01-01,NaN,...,NaN,PQT/MB/12 doses,12.0,NaN,NaN,3.0,2016-02-05,Cure,NaN,38.0
4,2,A309,2014-01-02,2014,29.0,291735,1403.0,6352960.0,2013-12-27,NaN,...,Zero Degree,PQT/MB/12 doses,12.0,NaN,NaN,8.0,2015-02-20,Cure,NaN,52.0


In [12]:
def count_death(x):
    if x == "Non death":
        return 0
    elif x == "Death":
        return 1
df["death"] = df["tpalta_n"].apply(lambda x: count_death(x))

In [13]:
df["cases"] = 1
df["year"] = pd.DatetimeIndex(df["dt_notific"]).year
#df["month"] = pd.DatetimeIndex(df["dt_notific"]).month
#df["day"] = 1
data_to_model = df[["id_mn_resi","cases", "year"]].groupby(["year", "id_mn_resi"])["cases"].sum().reset_index()
data_to_model.head()

,year,id_mn_resi,cases
0,2011,110001.0,21
1,2011,110002.0,47
2,2011,110004.0,67
3,2011,110005.0,16
4,2011,110006.0,4


In [14]:
data_to_model.to_csv("../data/2023_02_10_Data_to_analyze.csv", index = False)